## Import Packages for Data Manipulation, Graphing, LDA, and stats

In [4]:
import numpy as np
import pandas as pd
import re

#plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.grid_objs import Grid, Column #sliders
import plotly.graph_objs as go #horizontal bar charts
import plotly.plotly as py
from plotly import tools #side by side
init_notebook_mode(connected=True)

#LDA
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
stop = stopwords.words('english')
pyLDAvis.enable_notebook()
from gensim import corpora, models

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

## Read in the files

In [3]:
tweets = pd.read_csv('./finalprojdata/tweets.csv')
users = pd.read_csv('./finalprojdata/users.csv')
fulltweets = pd.read_csv('./finalprojdata/fulltweets2.csv')

## Data Processing and Cleaning

In [5]:
#clean the date to a Year-Month format
users['Date'] = pd.to_datetime(users['created_at'])
users = users[pd.notnull(users['created_at'])]
users = users.drop_duplicates(subset=['id'])
users['Date'] = users['Date'].apply(lambda x: x.strftime('%Y-%m'))

## Get the "names" of the fake accounts

In [7]:
usersname = pd.DataFrame(users.name.str.split(' ',1).tolist(),
                                   columns = ['first','last'])

usersnamesum = usersname.groupby('first',as_index=False).size().reset_index(name='counts')

usersnamesum = usersnamesum.sort_values('counts', ascending=False).head(20)

## Plot 1: Names - First and Last

In [8]:
## side by side bar plot
#first names
firstnames = usersname.groupby('first',as_index=False).size().reset_index(name='counts')
firstnames = firstnames.sort_values('counts', ascending=False).head(20)
data1 = go.Bar(
            x=firstnames['counts'],
            y=firstnames['first'],
            orientation = 'h',
            name = 'First Name'
)

#lastnames
lastnames = usersname.groupby('last',as_index=False).size().reset_index(name='counts')
lastnames  = lastnames .sort_values('counts', ascending=False).head(20)
data2 = go.Bar(
            x=lastnames ['counts'],
            y=lastnames ['last'],
            orientation = 'h',
            name = 'Last Name'
)

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('First Name','Last Name'))
fig.append_trace(data1, 1, 1)
fig.append_trace(data2, 1, 2)
fig['layout'].update(height=600, width=800, title='Totals of First and Last Names of Fake Accounts', titlefont=dict(size=25))
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



## Plot 2: Topical Sentiment of User Profile Description

In [6]:
#get rid of the special characters
descrip = users.description.copy().astype(str)
descrip = descrip.str.replace('[^\w\s]','')
descrip = descrip.str.replace('[\\r|\\n|\\t|_]',' ')
descrip = descrip.str.strip()

fulltweets_descrip = fulltweets.copy()
fulltweets_descrip.descrip = descrip
fulltweets_descrip.descrip = fulltweets_descrip.descrip.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop)]))

In [7]:
# compile sample documents into a list
doc_set = fulltweets_descrip.descrip.values.copy()
# loop through document list
texts = [text.split(' ') for text in doc_set]
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary)

In [8]:
data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
15     5.455422        1       1 -0.169745 -0.084908
5      5.234290        1       2 -0.084177 -0.040245
17     4.993329        1       3  0.089558  0.036883
14     4.988085        1       4 -0.015346 -0.137428
23     4.672318        1       5  0.056030 -0.016787
12     4.467955        1       6 -0.059535  0.104329
7      4.172408        1       7 -0.061934  0.136653
3      3.900915        1       8 -0.068258  0.080891
6      3.800262        1       9 -0.124044  0.050364
21     3.706461        1      10  0.038164  0.019108
24     3.544598        1      11  0.067578 -0.037070
27     3.403240        1      12  0.090698  0.013525
2      3.366910        1      13  0.049272 -0.005665
28     3.284541        1      14  0.053983 -0.031661
9      3.281234        1      15 -0.107975  0.091047
16     3.207199        1      16 -0.078880 -0.040377
26     3.180464        1      17  0.035349 -0.054737
8      3.091445        1      18  0.062214  0.019102
19     2.939099        1      19 -0.054427 -0.081832
22     2.873742        1      20  0.042373 -0.030356
20     2.773903        1      21  0.061957  0.011702
25     2.655947        1      22 -0.107650 -0.093424
18     2.542528        1      23  0.065388  0.000265
11     2.520494        1      24  0.065208  0.005424
10     2.414455        1      25  0.051294  0.011851
29     2.132787        1      26  0.025019  0.001283
13     2.113424        1      27  0.048764  0.006835
4      1.888520        1      28  0.017768 -0.010255
1      1.811615        1      29 -0.029986  0.057915
0      1.582408        1      30  0.041339  0.017566, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
24    Default  26.000000               nan  26.000000  30.0000  30.0000
39    Default  34.000000              news  34.000000  29.0000  29.0000
1465  Default   6.000000                HA   6.000000  28.0000  28.0000
585   Default  10.000000                 в  10.000000  27.0000  27.0000
886   Default  16.000000                us  16.000000  26.0000  26.0000
646   Default  12.000000                 и  12.000000  25.0000  25.0000
4     Default   8.000000            people   8.000000  24.0000  24.0000
880   Default  14.000000                DM  14.000000  23.0000  23.0000
53    Default  10.000000                Im  10.000000  22.0000  22.0000
153   Default  18.000000      Conservative  18.000000  21.0000  21.0000
911   Default   9.000000          Breaking   9.000000  20.0000  20.0000
104   Default   8.000000               God   8.000000  19.0000  19.0000
44    Default  13.000000  BlackLivesMatter  13.000000  18.0000  18.0000
28    Default  10.000000                не  10.000000  17.0000  17.0000
59    Default   7.000000              love   7.000000  16.0000  16.0000
305   Default   6.000000             Black   6.000000  15.0000  15.0000
474   Default   8.000000            sports   8.000000  14.0000  14.0000
465   Default   8.000000          politics   8.000000  13.0000  13.0000
41    Default   6.000000              dont   6.000000  12.0000  12.0000
916   Default   6.000000      endorsements   6.000000  11.0000  11.0000
915   Default   6.000000           anytime   6.000000  10.0000  10.0000
918   Default   6.000000           weather   6.000000   9.0000   9.0000
913   Default   6.000000               RTs   6.000000   8.0000   8.0000
917   Default   6.000000           traffic   6.000000   7.0000   7.0000
127   Default   9.000000              tcot   9.000000   6.0000   6.0000
189   Default   7.000000             PJNET   7.000000   5.0000   5.0000
220   Default   5.000000             world   5.000000   4.0000   4.0000
176   Default   6.000000     WakeUpAmerica   6.000000   3.0000   3.0000
145   Default   6.000000              TCOT   6.000000   2.0000   2.0000
611   Default   6.000000     

## Plot 3: When were the accounts made?

In [9]:
# group by Date, create a count and sort
userssum = users.groupby('Date',as_index=False).size().reset_index(name='counts')
userssum = userssum.sort_values('Date')

In [46]:
trace0 = Bar(
    name = "Accounts Created Over Time",
    x=userssum.Date,
    y=userssum.counts)
data = Data([trace0])

layout = Layout(
    title = "Accounts Made from 2009 to 2017",
    yaxis=dict(
        title='Number of Accounts Made',
        range=[0, 100],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2009-01','2017-1'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig)

## Not plotted because of too much missing data: Where are they from?

In [65]:
users.location.value_counts()

USA                              93
United States                    23
Москва                           16
Москва, Россия                    9
Chicago, IL                       7
Atlanta                           6
US                                6
Atlanta, GA                       5
Houston, TX                       3
Texas, USA                        3
New York, NY                      3
Baltimore, MD                     3
Санкт-Петербург                   3
Los Angeles, CA                   3
Washington, DC                    2
Мурманск                          2
NY                                2
Омск                              2
St Louis, MO                      2
Baton Rouge, LA                   2
Philadelphia, PA                  2
Miami, FL                         2
New York, USA                     2
the corner of happy & healthy     1
Чита                              1
NYC                               1
Brussel, België                   1
Richmond, VA                

## Plot 4: How influential are these users

In [42]:
#create a line of best fit
slope, intercept, r_value, p_value, std_err = stats.linregress(users.followers_count,users.statuses_count)
line = slope*users.followers_count+intercept

In [44]:
trace = go.Scatter(
    x = users.followers_count,
    y = users.statuses_count,
    mode = 'markers',
    text = users.name,
    marker = dict(size = 10,
        color = 'blue',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        ))
)

trace2 = go.Scatter(
                  x=users.followers_count,
                  y=line,
                  mode='lines',
                  marker=go.Marker(color='black'),
                  name='Fit'
                  )

data = [trace, trace2]

layout = Layout(
    title = "Russian Fake Account Followers and Tweet Count",
    xaxis=dict(
            title='Number of Followers',
            dtick=10000,
            range=[0, 100000],
            titlefont=dict(
            size=20,
        )
    ),
   
    yaxis = dict(
        title='Number of Tweets',
        range=[0, 70000],
        titlefont=dict(
            size=20,
        )
    ),
    
    annotations=[dict(y=20576,x=61609,xref='x',
            yref='y',text='Jenna Abrams'),
                dict(y=42438,x=60897,xref='x',
            yref='y',text='New York City Today'),
                dict(y=44882,x=27745,xref='x',
            yref='y',text='New Orleans Online'),
                dict(x=98412,y=30624,xref='x',
            yref='y',text='Максим Дементьев'),
                dict(y=1370,x=11518,xref='x',
            yref='y',text='Black News'),
                dict(y=57212,x=11370,xref='x',
            yref='y',text='РИА ФАН'),
                dict(y=28939,x=31729,xref='x',
            yref='y',text='Washington Online')]
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(height=900, width=1200, titlefont=dict(size=20))
iplot(fig)

## Plot 5: When are they posting?

In [13]:
# heat map showing the days and hours of users
m = pd.pivot_table(fulltweets,values = 'user_key',index='created_strDayofweek',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)
z= m.as_matrix()

In [70]:
trace = Heatmap(z=z,
                x=[i for i in np.arange(0,24)],
                y=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets Per Day Per Month',
    xaxis=dict(
        nticks=24, 
        title = 'Month',
        titlefont=dict(
        size=20)),
    
    yaxis = dict(
    ),

)
data=[trace]
fig = Figure(data=data,layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig, filename='numberoftweets.html')

## Plot 6: What are they saying?

In [17]:
# LDA topics of actual tweets
dictionary = gensim.corpora.Dictionary.load('dictionary.dict')
corpus = gensim.corpora.MmCorpus('corpus.mm')
ldamodel = gensim.models.LdaModel.load('topic.model')

In [18]:
data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     5.652078        1       1  0.269390 -0.038418
21     4.082652        1       2  0.207074 -0.011219
28     4.003842        1       3 -0.044191  0.224570
27     3.965532        1       4  0.087466  0.131443
9      3.850252        1       5  0.190262  0.006419
25     3.680442        1       6  0.166511 -0.017891
11     3.526953        1       7 -0.027825  0.066462
14     3.453641        1       8  0.083642 -0.104792
24     3.438111        1       9 -0.027985  0.111117
4      3.321473        1      10 -0.078400  0.164542
0      3.231427        1      11 -0.017994 -0.076229
13     3.230222        1      12  0.058620 -0.002203
23     3.223801        1      13 -0.023846  0.128801
12     3.188437        1      14 -0.003502  0.105461
7      3.184742        1      15 -0.043826 -0.092018
19     3.098035        1      16 -0.062619 -0.077927
8      3.086067        1      17 -0.027137 -0.051963
29     3.080491        1      18 -0.049124 -0.019002
3      3.076198        1      19  0.050652  0.003056
16     3.033868        1      20 -0.097865 -0.055391
10     3.015934        1      21 -0.080050 -0.095464
22     3.012110        1      22 -0.038976 -0.059701
20     3.003440        1      23 -0.085357 -0.049674
5      2.999063        1      24 -0.072599 -0.006369
15     2.984936        1      25 -0.050851 -0.035763
1      2.951488        1      26 -0.080618 -0.057125
18     2.946492        1      27 -0.084379  0.010517
2      2.922456        1      28 -0.059199 -0.082429
6      2.913232        1      29 -0.059057  0.003384
26     2.842588        1      30  0.001783 -0.022193, topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
235874  Default  23817.000000            trump  23817.000000  30.0000  30.0000
31953   Default   9612.000000          clinton   9612.000000  29.0000  29.0000
172447  Default   9021.000000          hillary   9021.000000  28.0000  28.0000
38966   Default   4989.000000             dont   4989.000000  27.0000  27.0000
48151   Default   3588.000000             tcot   3588.000000  26.0000  26.0000
56046   Default   4473.000000           donald   4473.000000  25.0000  25.0000
27477   Default   3921.000000         politics   3921.000000  24.0000  24.0000
183588  Default   3521.000000          america   3521.000000  23.0000  23.0000
10757   Default   4863.000000             like   4863.000000  22.0000  22.0000
36914   Default   3093.000000        president   3093.000000  21.0000  21.0000
238384  Default   2893.000000             know   2893.000000  20.0000  20.0000
135144  Default   2797.000000             vote   2797.000000  19.0000  19.0000
22149   Default   2611.000000            never   2611.000000  18.0000  18.0000
233129  Default   2505.000000             good   2505.000000  17.0000  17.0000
119324  Default   2670.000000             want   2670.000000  16.0000  16.0000
223723  Default   2392.000000            white   2392.000000  15.0000  15.0000
136538  Default   2599.000000            pjnet   2599.000000  14.0000  14.0000
74981   Default   2432.000000             love   2432.000000  13.0000  13.0000
66034   Default   2584.000000             maga   2584.000000  12.0000  12.0000
176039  Default   2247.000000         midnight   2247.000000  11.0000  11.0000
231205  Default   2439.000000            think   2439.000000  10.0000  10.0000
7915    Default   2240.000000             cant   2240.000000   9.0000   9.0000
136102  Default   7366.000000            obama   7366.000000   8.0000   8.0000
30154   Default   2033.000000             back   2033.000000   7.0000   7.0000
35015   Default   1938.000000         american   1938.000000   6.0000   6.0000
190213  Default   4277.000000             news   4277.000000   5.0000   5.0000
216484  Default   2735.000000             make   

## How are they gaining attention?

In [19]:
subusers = fulltweets.groupby('user_key', as_index=False).mean().sort_values(by='followers_count', ascending=False).user_key[0:20].values

In [20]:
#create monthly aggregated data
subfulltweets2 = fulltweets[fulltweets.user_key.isin(subusers)][['user_key', 'followers_count', 'Subjectivity', 'Sentiment', 'statuses_count', 'created_str']]
subfulltweets2['Date'] = pd.to_datetime(subfulltweets2['created_str'])
subfulltweets2 = subfulltweets2[pd.notnull(subfulltweets2['created_str'])]
subfulltweets2['Date'] = subfulltweets2['Date'].apply(lambda x: x.strftime('%Y-%m')) #change to '%Y-%m-%d' for fine tuned

## Plot 7: Tweet Velocity

In [22]:
tweetcount = subfulltweets2.groupby('Date',as_index=False).size().reset_index(name='counts')
tweetcount = tweetcount.sort_values('Date', ascending=True)


In [47]:
trace0 = Scatter(
    name = "Tweets Made",
    x=tweetcount.Date,
    y=tweetcount.counts,
    line = dict(
        width = 4))
data = Data([trace0])

layout = Layout(
    title = "Number of Tweets by Top 20 Followed Fake Accounts",
    yaxis=dict(
        title='Number of Tweets',
        range=[0, 3500],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig)

## Plot 8: Daily Totals

In [73]:
#create daily aggregated data
subfulltweets3 = fulltweets[fulltweets.user_key.isin(subusers)][['text','user_key', 'followers_count', 'Subjectivity', 'Sentiment', 'statuses_count', 'created_str']]
subfulltweets3['Date'] = pd.to_datetime(subfulltweets3['created_str'])
subfulltweets3 = subfulltweets3[pd.notnull(subfulltweets3['created_str'])]
subfulltweets3['Date'] = subfulltweets3['Date'].apply(lambda x: x.strftime('%Y-%m-%d')) #change to '%Y-%m-%d' for fine tuned
tweetcount2 = subfulltweets3.groupby('Date',as_index=False).size().reset_index(name='counts')
tweetcount2 = tweetcount2.sort_values('Date', ascending=True)

In [98]:
trace0 = Scatter(
    name = "Tweets Made",
    x=tweetcount2.Date,
    y=tweetcount2.counts,
    line = dict(
        width = 2))
data = Data([trace0])

layout = Layout(
    title = "Daily Tweets by Top 20 Followed Fake Accounts",
    yaxis=dict(
        title='Number of Tweets',
        range=[0, 600],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    ),
    annotations=[dict(y=484,x='2016-10-17',xref='x',
            yref='y',text='#makemehateyouinonephrase'),
                dict(y=113,x='2016-08-04',xref='x',
            yref='y',text='#obamaswishlist'),
                dict(y=301,x='2016-12-07',xref='x',
            yref='y',text='#idrunforpresidentif'),
                dict(y=374,x='2016-10-05',xref='x',
            yref='y',text='#ruinadinnerinonephrase'),
                dict(y=330,x='2017-01-02',xref='x',
            yref='y',text='#survivialtips')]
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig)

In [9]:
#subfulltweets3[subfulltweets3['Date']=='2017-01-02'] #use to look at inidivdual dates

In [24]:
tweetsent = subfulltweets2.groupby('Date',as_index=False)['Sentiment','Subjectivity'].agg(np.mean)
tweetsent = tweetsent.sort_values('Date', ascending=True)

## Plot 9: Combined Graphs of Subjectivity and Sentiment

In [35]:
trace0 = Scatter(
    name = "Sentiment",
    x=tweetsent.Date,
    y=tweetsent.Sentiment,
    line = dict(
        width = 3))

trace1 = Scatter(
    name = "Subjectivity",
    x=tweetsent.Date,
    y=tweetsent.Subjectivity,
    line = dict(
        width = 3))

data = Data([trace0, trace1])

layout = Layout(
    title = "Subjectivity and Sentiment of Top 20 Fake Accounts",
    yaxis=dict(
        title='Value',
        range=[-1,1],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig)